## Analyzing CIA Factbook Data (SQL)

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

### Overview of Data

In [4]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


### Exploring the Data 

In [5]:
%%%sql
SELECT * 
    FROM facts
 Limit 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The columns provided in the fact table include:
1. `name` - The name of the country 
2. `area` - The country's total area (land and water)
3. `area_land` - Land area in Kilometers
4. `area_water` - Water area in kilometers
5. `population` - Country's population 
6. `population_growth` - Country's population in percentage
7. `birth_rate` - the country's birth rate per 1,000 people
8. `death_rate` - the country's death rate per 1,000 people

To begin exploring the data, I will compute a handful of summary statistics

In [6]:
%%%sql
SELECT MAX(population) as Max_Population,
        MIN(population) as Min_Population,
        MAX(population_growth) as Max_Population_Growth,
        MIN(population_growth) as Min_Population_Growth
    FROM facts

Done.


Max_Population,Min_Population,Max_Population_Growth,Min_Population_Growth
7256490011,0,4.02,0.0


### Exploring Outliers

In [10]:
%%%sql
SELECT name,population 
    FROM facts 
    WHERE population = (select MAX(population) FROM facts)
    or population = (select MIN(population) FROM facts)

Done.


name,population
Antarctica,0
World,7256490011


To find the outliers of the table, I reran a query to identify the country whose population was the maximum and minimun. It turns out the maximum population is the total World population, and the minimum is the population of Antartctica. 

In [14]:
%%%sql
SELECT  MAX(population) as Max_Population,
        MIN(population) as Min_Population,
        MAX(population_growth) as Max_Population_Growth,
        MIN(population_growth) as Min_Population_Growth
    FROM facts 
    WHERE name != 'World'

Done.


Max_Population,Min_Population,Max_Population_Growth,Min_Population_Growth
1367485388,0,4.02,0.0


### Exploring Population and Area Averages

In [15]:
%%%sql 
SELECT AVG(Population) as 'Average Population', 
        AVG(area) as 'Average Area'
    FROM facts
    WHERE name !='World'

Done.


Average Population,Average Area
32242666.56846473,555093.546184739


In [18]:
%%%sql 
SELECT name as 'Country',population,area
    FROM facts
    WHERE population > (SELECT AVG(population) FROM facts)
    AND area < (SELECT AVG(area) FROM facts)

Done.


Country,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Japan,126919659,377915
Philippines,100998376,300000
Thailand,67976405,513120
United Kingdom,64088222,243610
Vietnam,94348835,331210
